# PyEO Machine Learning: How to train your classifier..

- Here we will create a model to classify land cover (and thus enable the detection changes in forest cover between two date periods).
- To build a model, we will need training data. For PyEO, we define training data as pixel surface reflectance values over multiple wavebands and their corresponding 'true' land cover classes to which they should ideally be assigned by the model
- The next section below details how to generate this training data by manually delineating polygons using the QGIS software package

# Delineate Sample Areas using QGIS

## Live Demo: Training Sample Delineation over Mau Forest

- Prof. Heiko Balzter will show a live demo of how to delineate training data in QGIS.

## Instructional Video: Training sample delineation over Rutland, UK

- Additionally, a recording of a previous demo is included with the files we distributed to attendees earlier, to view at your leisure.
    - Alternatively, use this [link](https://drive.google.com/file/d/1d-eaZ3iOcruZyv35G3wRqHakUWiMoJl-/view?usp=sharing]) to watch the video Google Drive

## QGIS Training Data Specification Process Guide
- In the left hand panel of JupyterLab, navigate to `pyeo/models` within your installation directory and right click to download the sample image to your laptop
  - You need the file `composite_T36MYE_20230220T074941_clipped.tif`
- Once downloaded, open the image in QGIS on your laptop
- Add a shapefile layer
- Toggle editing of the shapefile layer
- Add a field which should be integer and named `class`
- Delineate features using the **Add Polygon Feature** tool. It looks like this: ![image](./Create_polygon.PNG) 
- Add training polygons for classes of interest to your shape file. 
- So that we can compare results please train for the following land use classes, using the integer CODE assigned to each in the table below:
  - `Primary Forest` 1; 
  - `Plantation Forest` 2;
  - `Bare Soil` 3;
  - `Crops` 4;
  - `Grassland` 5;
  - `Artificial` 13;
- Save your shapefile to your laptop hard drive
- Use the 'Upload' button to bring your saved shapefile to the folder 'model_dir' within your PyEO installation on SEPAL


## QGIS Tips

- Whilst in QGIS, make sure you have the **Digitizing** and **Advanced Digitizing** toolbars enabled. To do so, click:  
    - View > Toolbars > Tick Digitizing and Advanced Digitizing
- Avoid delineating near the edges of land cover, e.g. at the forest edge adjacent to a road. These edge pixels will capture the variation in land-cover at the sub-pixel level, confusing your classifier. 
- The **Reshape Features** tool, is extremely useful for extending already delineated polygons. Click inside the polygon you want to extend, then click any points outside of the polygon (covering your land cover class of interest), once ready - click inside the polygon to complete the loop, and right click to finish delineation. The tool icon looks like this: ![image](./Reshape_features.PNG)
- Install Google Earth Pro on your PC, https://www.google.com/earth/about/versions/ , which offers very-high resolution satellite imagery, and Google StreetView, to help you understand the land covers you are delineating.
- Install the QGIS plugin, **Send2GE**. This plugin sends your location directly to your installed Google Earth Pro. To do so, click on the Send2GE icon, ![image](./Send2GE.PNG), and then click on the location you want to view in Google Earth Pro.
- When developing models in future you may wish to develop your own choice of Land Cover classes and tailor them to what you want your model to achieve. Listed below is the full list of classes that we have evaluated for our `PyEO` forest alert classifier, but alternative classes may be more appropriate for your needs.

    1. Primary Forest
    2. Plantation Forest
    3. Bare Soil
    4. Crops
    5. Grassland
    6. Open Water
    7. Burn Scar
    8. Cloud
    9. Cloud Shadow
    10. Haze
    11. Sparse Woodland
    12. Dense Woodland
    13. Artificial
    

# Setup: Requirements to use this Notebook

## Select the virtual environment
- Use the drop-down list at the top right of the Jupyter notebook window
- Select `(venv) Python for Earth Observation (PyEO)`

## Check the working directory is set to `pyeo` within your PyEO installation

In [ ]:
pwd

In [ ]:
cd /home/sepal-user/20230626_pyeo_installation/pyeo

In [ ]:
pwd

# Import the Required Libraries

Before proceeding, let's check that the import statements from pyeo execute correctly.

In [ ]:
import os
from pathlib import Path
from pyeo import classification as cls
from pyeo import filesystem_utilities as file_utils
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import ensemble as ens
import numpy as np
import joblib

print("Library imports complete")

# Specify the Model Storage Directory

In [ ]:
pyeo_dir = "/home/sepal-user/20230626_pyeo_installation/pyeo"

In [ ]:
os.chdir(pyeo_dir)

In [ ]:
pwd

In [ ]:
model_dir = "./models/my_model"
log_dir = "./log"

# Create the folder if it doesn't yet exist
os.chdir(pyeo_dir)
if not os.path.exists(model_dir): 
    os.mkdir(model_dir)
if not os.path.exists(log_dir): 
    os.mkdir(log_dir)
    

- Use the File Browser to check your new folder now exists

# Initialise Logging

- When running `PyEO`, we record our processing efforts, parameters etc. in a log file (a `.log` file).
- If we are ever curious about what `PyEO` has done and when, we can look at the log file.  
- Below, we initialise our log file 
- **Note: the .log does not need to exist before running the function**

In [ ]:
log_filename = "model_log.log"
log = file_utils.init_log(os.path.join(log_dir, log_filename))

# Let's add some information to the log
log.info(f'pyeo_Model_Training Notebook Started:')
log.info(f'- pyeo_dir set to: {pyeo_dir}')
log.info(f'- model_dir set to: {model_dir}')
log.info(f'- log_dir set to: {log_dir}')

- Whenever a function writes to the log, the output will be displayed in the Python cell output, but the log file can also be viewed in a text editor.  
- **Right-Click on model_log.txt in the JupyterLab explorer to the left and select 'open' to view in a tab within JupyterLab.**   

# Model Training and Classification, Step by Step

- In this section, we will follow the individual steps to extract spectral information from the Sentinel-2 imagery using our training shapefiles, train a Random Forest model on the spectral values of the land cover classes, and classify a Sentinel-2 image tile.

## Extract Spectral Information from Sentinel-2 Imagery

- Since we have delineated our training polygons, we can now extract the spectral information from the imagery, by running `cls.extract_features_to_csv()` in the cell below.  
- First, specify the paths to the training shapefile, training raster and the out directory.

In [ ]:
# specify the path of the training shapefile
training_shape_filename = "composite_T36MYE_20230220T074941_clipped_features.shp"
# specify the path of the training raster
training_raster_filename = "composite_T36MYE_20230220T074941_clipped.tif"
# specify the output folder
out_dir = model_dir
# specify the output filename for the model
model_name = "my_model"
model_out_filename = f"{model_name}.pkl"
# specify the output filename for the csv
feature_out_filename = f'{model_name}_features.csv'

log.info(f'File and Folder Specification Started::')
log.info(f'- training_shape_filename set to: {training_shape_filename}')
log.info(f'- training_raster_filename set to: {training_raster_filename}')
log.info(f'- out_dir set to: {out_dir}')
log.info(f'- model_out_filename set to: {model_out_filename}')
log.info(f'- feature_out_filename set to: {feature_out_filename}')


### Create `out_folder` if it does not exist

In [ ]:
# if not os.path.exists(os.path.join(pyeo_dir, out_dir)):
    # os.mkdir(os.path.join(pyeo_dir, out_dir))

### Run `extract_features_to_csv`

Now, we can run the `cls.extract_features_to_csv` function.  
If the column heading of your training shapefile is not `CODE`, substitute the correct column name with the `attribute` argument.

In [ ]:
# declare our path variables as easier to read variables
out_path = os.path.join(out_dir, feature_out_filename)
training_raster_path = os.path.join(model_dir, training_raster_filename)
training_shapefile_path = os.path.join(model_dir, training_shape_filename)

log.info(f'Feature Extraction Started:')
log.info(f'- out_path set to: {out_path}')
log.info(f'- training_raster_path set to: {training_raster_path}')
log.info(f'- training_shapefile_path set to: {training_shapefile_path}')

In [ ]:
# run the function to extract the spectral values and assigned classes from the training raster and training shapefile
cls.extract_features_to_csv(in_ras_path=training_raster_path,
                            training_shape_path=training_shapefile_path,
                            out_path=out_path,
                            attribute="class")

- Let's see what `features.csv` looks like, by reading in the csv and printing the first 5 rows.  
- Each row represents a pixel of a certain class. The first column in each row is the class of that pixel, with the columns after that being the Sentinel 2 blue, green, red and NIR value of that pixel.

In [ ]:
# declare out_path from previous cell as csv_path, to avoid confusion
csv_path = out_path

# read in our spectral signatures
features = pd.read_csv(csv_path)

features.columns = ["class", "R", "G", "B", "NIR"]

# prints the first 5 rows of features
features.head(5)

In [ ]:
features.tail(5)

- Note: your model folder should now contain a .csv file of features - check using the file explorer on the left

## Model Creation and Training

- Now we can create a scikit-learn Extra Trees Classifier using PyEO:

In [ ]:
log.info(f'Model Building Started:')

# initialise an Extra Trees Classifier
model = ens.ExtraTreesClassifier(bootstrap=False, criterion="gini", max_features=0.55, min_samples_leaf=2,
                                    min_samples_split=16, n_estimators=100, n_jobs=-1, class_weight='balanced')

# load signatures from csv file
features, labels = cls.load_signatures(csv_path, sig_datatype=np.int32)

# split into test and train samples, reserving 25% for testing model accuracy
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=.25, random_state=101)

# train the model
model.fit(X_train, y_train)

# save the model
model_path = os.path.join(out_dir, model_out_filename)
joblib.dump(model, filename = model_path)

log.info(f"Model saved at {model_path}")

- Note: your model folder should now contain a .pkl file containing your model - check using the file explorer on the left

## Accuracy Statistics

- Now that we have a model trained from shapefiles defined over Sentinel-2 image tile, we should assess the model accuracy.  
- To do that, we will use the model to predict the classes of `y_test` from the values of `X_test`, which comprise the 25% that we reserved for testing model accuracy.

In [ ]:
log.info(f'Model Performance Analysis Started:')

confusion_matrix_out_filename = f'{model_name}_confusion_matrix.txt'
class_scores_out_filename = f'{model_name}_class_scores.txt'

# make a prediction
y_pred = model.predict(X_test)

# create a confusion matrix
matrix = confusion_matrix(y_true=y_test, y_pred=y_pred,)

# write the confusion matrix to file
with open(os.path.join(out_dir, confusion_matrix_out_filename), "w") as txt:
    print(matrix, file=txt)

# write classification labels
target_names = ["Primary Forest",
          "Plantation Forest",
          "Bare Soil",
          "Crops",
          "Grassland",
          "Artificial"]

# view the matrix
print(matrix)

In [ ]:
# create classification metrics report
report = classification_report(y_true=y_test, y_pred=y_pred, target_names=target_names, digits=6)

# write classification report to file
with open(os.path.join(out_dir, class_scores_out_filename), "w") as txt:
    print(report, file=txt)

# view the report
print(report)

## Image Classification

- Once we are happy with the performance of the model we can now classify a Sentinel-2 image tiles so long as they have similar land use and vegetation cover (and thus spectral characteristics) to the imagery used to train the model.
- Here, we will simply use our training image and see how well the classification extends over the untrained areas of the image

In [ ]:
log.info(f'Image Classification with Model Started:')

# test_raster_path = training_raster_path
test_raster_filename = 'composite_T36MYE_20230220T074941_clipped.tif'
test_raster_path = os.path.join(model_dir, test_raster_filename)

class_out_filename = f'{os.path.splitext(test_raster_filename)[0]}_full_class.tif'
class_out_path = os.path.join(out_dir, class_out_filename)

cls.classify_image(image_path=test_raster_path,
                   model_path=model_path,
                   class_out_path=class_out_path,
                   prob_out_path=None,
                   apply_mask=False,
                   out_format="GTiff",
                   chunks=10,
                   nodata=0,
                   skip_existing = False)

log.info(f'Classified Image Saved: {class_out_path}')


- Note: your model folder should now contain a classified image with '_full_class' appended to the raw image name - check using the file explorer on the left

## Visualise the Classified Imagery

Now we can now define a function to colormap and visualise the classified image

In [ ]:
## Visualise the Classified Imagery

from matplotlib.colors import LinearSegmentedColormap
from osgeo import gdal
import warnings
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

log.info(f'Classified Image Visualisation function has been defined.')

def visualise_classification(classified_path: str, labels: list):

    ds = gdal.Open(classified_path)
    array = np.array(ds.GetRasterBand(1).ReadAsArray())
   
    unique, _counts = np.unique(array, return_counts=True)

    counts=list(np.zeros(max(unique)+1,'int'))
    for i in range(0, len(_counts)):
        counts[unique[i]] = _counts[i]

    #counts, _ = np.histogram(array, bins = len(labels))
    print(counts)
   
    percent = counts / sum(counts)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (12, 6))


    def discrete_cmap(N, base_cmap=None):
        """Create an N-bin discrete colormap from the specified input map"""

        # this function was taken on 10/03/2023 from https://gist.github.com/jakevdp/91077b0cae40f8f8244a
        # author: Jake Vanderplas
        
        # warnings.filterwarnings("ignore", category=MatplotlibDeprecationWarning)
        base = plt.cm.get_cmap(base_cmap)
        color_list = base(np.linspace(0, 1, N))
        cmap_name = base.name + str(N)
        return base.from_list(cmap_name, color_list, N)

    cmap = discrete_cmap(len(labels), base_cmap="coolwarm")
    # cmap = discrete_cmap(len(labels), base_cmap="cubehelix")
    # cmap.set_bad("red")
    colour_list = [mcolors.rgb2hex(cmap(i)) for i in range(cmap.N)]
    if cmap.N <=14:
        log.info("Defining custom colour table for up to 13 classes and a missing data class of 0 (0..13)")
        colour_list = [
                      [0, 0, 0, 1],  # no data
                      [0, 100/255, 0, 1],  # Primary Forest
                      [154/255, 205/255, 50/255, 1],  # plantation Forest
                      [139/255, 69/255, 19/255, 1],  # Bare Soil
                      [189/255, 183/255, 107/255, 1],  # Crops
                      [240/255, 230/255, 140/255, 1],  # Grassland
                      [0, 0, 205/255, 1],  # Open Water
                      [128/255, 0, 0, 1],  # Burn Scar
                      [255/255, 255/255, 255/255, 1],  # cloud
                      [60/255, 60/255, 60/255, 1],  # cloud shadow
                      [128/255, 128/255, 128/255, 1],  # Haze
                      [46/255, 139/255, 87/255, 1],  # Open Woodland
                      [92/255, 145/255, 92/255, 1],  # Closed Woodland
                      [255/255, 30/255, 30/255, 1]  # Artificial
                      ]
    # Create the colormap
    n_bins = len(labels)
    cmap_name = 'my_colour_map'
    cmap = LinearSegmentedColormap.from_list(cmap_name, colour_list, N=n_bins)
   
    x = np.arange(0, len(labels))
    ax1.bar(x, percent, color=colour_list)
    ax1.set_title("Classes distribution")
    ax2.set_title("Classification raster displayed")
    ax2.imshow(array, cmap=cmap, aspect="auto", vmin=-0.5, vmax=len(labels)+0.5)
    plt.show()

    return


In [ ]:
# create a list of class labels

labels = [
    "No Data",
    "Primary Forest",
    "Plantation Forest",
    "Bare Soil",
    "Crops",
    "Grassland",
    "Open Water",
    "Burn Scar",
    "Cloud",
    "Cloud Shadow",
    "Haze",
    "Open Woodland",
    "Closed Woodland",
    "Artificial"
    ]

visualise_classification(classified_path=class_out_path, labels=labels)

# Extended Model Training and Classification
- **NOTE: These PyEO capabilities are beyond the tutorial's scope but illustrative code and outputs are included below as a guide to their use**

## Model Building from Multiple Tiles and Shapefiles

- PyEO can also train a Random Forest classifier on multiple Sentinel-2 tiles with multiple training shapefiles. This approach can create a superior classifier because the training shapefiles can cover more tiles, providing the model with a variety of spectral values per class, and thus show greater versatility over a wide range of seasonal land cover and variation.


The `create_rf_model_for_region` function assumes the training rasters and training shapefiles are structured like the cell diagram below.   
**Note, the filenames do not need to be a specific format**

The second function will we use is `classify_directory`, which classifies multiple images in one call.

In [ ]:
# model_folder
    # training_raster_tile_1
    # training_shapefile_tile_1
    # training_raster_tile_2
    # training_shapefile_tile_2

In [ ]:
# pyeo_home = "/data/clcr/shared/IMPRESS/Ivan/kenya_national_prod"

In [ ]:
# out_folder = "models/model_36MYE_37MER_37NCC_20230417"

In [ ]:
# # specify the path to the directory structure
# region_folder = os.path.join(pyeo_home, out_folder)

# # name the model that will be created
# model_file = "model_36MYE_37MER_37NCC_Unoptimised_20230417.pkl"
# model_path = os.path.join(region_folder, model_file)

# cls.create_rf_model_for_region(path_to_region=region_folder,
#                                model_out=model_path,
#                                attribute="class"
#                                #,attribute="CODE" # replace this line with the one above, if training shapefile has CODE as class heading
#                                )

In [ ]:
# view classification report metrics
# with open(f"{os.path.join(pyeo_home, out_folder)}{os.sep}class_scores.txt", "r") as report:
#     print(report.read())
    

## Classify Directory

- Another useful PyEO capability to be aware of is that classification (and other functions) can be automatically over a directory of images
- This provides the basis to build automated data pipelines that process image sets as a chain from directory to directory
- The code below shows how to classify a directory of images given its path, a random forest model to apply, and a destination folder for the results

`cls.classify_directory` requires the `class_out_dir` to already exist, so make sure that this directory is present before running the function.

In [ ]:
# provide the path of the directory of images to classify
# image_directory = pyeo_home

# # provide the path of the classified output folder
# class_out_dir = f"{os.path.join(pyeo_home, out_folder)}{os.sep}"

# # run the function to classify the directory of images you provided
# cls.classify_directory(in_dir=image_directory,
#                        model_path=model_path,
#                        class_out_dir=class_out_dir,
#                        chunks=10)

## Visualise Classifications

Here, we can visualise the images we have just classified. Just pass the filename of the classified image.

In [ ]:
# specify the directory of a classified image
# classified_path = f"{class_out_dir}/composite_T36NYF_20221202T075301_clipped_class.tif"

# # create a list of class labels. In this example we have more classes because the additional classes were delineated to train this region rf model.
# labels = [
#     "Primary Forest",
#     "Plantation Forest",
#     "Bare Soil",
#     "Crops",
#     "Grassland",
#     "Open Water",
#     "Artificial"
# ]

# visualise_classification(classified_path=classified_path, labels=labels)

In [ ]:
# # specify the directory of a classified image
# classified_path = f"{class_out_dir}/composite_T36MYE_20230220T074941_clipped_class.tif"

# visualise_classification(classified_path=classified_path, labels=labels)


**Question: Has this classifier done a better jo than than the simple one we created?**